# Clustering: K-Means

Nous présentons dans ce notebook l'algorithme de **clustering** dit des **K moyennes**( ou **K-Means**). C'est un **algorithme d'apprentissage non supervisé**.

Commençons par importer certaines libraires qui nous seront utiles:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')

## Présentation du K-Means

Le K Means est un algorithme de classification  **non supervisée**: cela signifie que le modèle cherche à comprendre la structure générale des données sans chercher à prédire une variable spécifiquement. 

Le K Means est assez intuitif, il cherche simplement à définir des centres de points proches dans les données. Le **centre** d'un **segment (cluster)**, va être défini comme étant la moyenne des points qui constitue le cluster. Pour attribuer  un cluster à un point, on cherche le centre qui est le plus proche du points

Réutilisons les données utilisées précédemment mais en supprimant les couleurs car nous ne disposons par de labelisation dans ce cas:

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=300, centers=4,
                  random_state=0, cluster_std=0.60)

plt.scatter(X[:, 0], X[:, 1], s=50);

Par simple observation, on identifie facilement que ces données peuvent être segmentés en quatre sous-ensembles homogènes. Si on cherche à calculer l'ensemble des segmentations de points possibles, on a un quatité exponetielle du nombre de points à évaluer. Cependant il existe une méthode de *maximisation de vraisemblance* efficace pour déterminer la segmentation optimale de manière plus rapide.

In [ ]:
from sklearn.cluster import KMeans
est = KMeans(4)  # 4 clusters
est.fit(X)
y_kmeans = est.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='rainbow');

L'algorithme segmente les points en 4 clusters de manière similaire à celle qu'un humain ferait par l'observation.

## L'algorihtme du K Means: maximisation de la vraisemblance

Le K-means utilise une méthode de maximisation de la vraisemblance pour dtéerminer une segmentation optimale. Cela consiste à:

1.Générer aléatoirement de centres de clusters

2.Répéter jusqu'à convergence:
    
    A.Regroupement des points au centre le plus proche
    B.Mise à jour des centres aux moyennes des clusters 

L'algorithme converge quand les centres stables après mise à jour. Les points restent figé dans leur segment.

Observons la convergence du K Means:

In [ ]:
from fig_code import plot_kmeans_interactive
plot_kmeans_interactive();

L'algorithme converge (presque) systématiquement vers la solution optimale.

### Limitation du K Means

La converge n'est pas garantie. Pour cette raison, la libraire scikit-learn utilise par défaut de multiples valuers d'initialisations des centre and conserve uniquement la meilleure solution.

De plus, le nombre de cluster K doit être fixé à priori. D'autres méthodes détermine une valuer optimale du nombre de clusters le plus adapté aux données.


## Exemple: clustering de chiffres manuscrits par le K Means

On s'intéresse à nouveaux aux données de chiffres manuscrits pour donner un exemple conret. Ici, nous utilisons le K Means pour regrouper automatiquement toutes les images qui représente un même chiffre. On va s'intéresser aux valeurs des centres obtenus par le K Means.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
est = KMeans(n_clusters=10)
clusters = est.fit_predict(digits.data)
est.cluster_centers_.shape

Nous obtenons 10 clusters de vecteurs en dimensions 64. Visualisons ces 10 centres de clusters pour comprendre ce qu'ils contiennent.

In [ ]:
fig = plt.figure(figsize=(8, 3))
for i in range(10):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    ax.imshow(est.cluster_centers_[i].reshape((8, 8)), cmap=plt.cm.binary)

**Sans disposer d'annotations à priori**, le K Means est en mesure de déterminer des cluters dont le centre correspondent visualement aux chiffres de 0 à 9 (sauf le 8!).

Les clusters ne sont par contre pas dans le bon ordre, on se propose de corriger cela:

In [ ]:
from scipy.stats import mode

labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]

Pour vérifier la pertinence de note clustering, évaluons la perfomrance de l'algorithme en classification.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(digits.target, labels)

80%, c'est un score honorable! La matrice de confusion nous permet d'évaluer plus en détails ces performances:

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(digits.target, labels))

plt.imshow(confusion_matrix(digits.target, labels),
           cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.grid(False)
plt.ylabel('true')
plt.xlabel('predicted');

Pour rappel, ce score de 80% précison en classification est obtenue **sans aucune supervision** de l'algorihtme, c'est à dire **aucune annotation préalable des données**.

## Autre exemple: KMeans pour la compression de couleurs


La compression de couleur consitute une application intéressante du clustering. Imaginons que l'on dispose d'une image avec potentiellment des millions de couleurs. Une majorité de ces couleurs va certainement être inutilisée par une image donnée. Et réciproquement, une même couleur peut être potentiellement utilisés par de nombreux pixels de l'image.

Scikit-learn propose des images à manipuler dans le module ``datasets`` :

In [ ]:
from sklearn.datasets import load_sample_image
china = load_sample_image("china.jpg")
plt.imshow(china)
plt.grid(False);

Cette image est stockée dans un array à 3 dimensions, de taille ``(height, width, RGB)``:


In [ ]:
china.shape

Cette image peut-être vue comme un suage de points dans une espace de couleurs à 3 dimensions. On normalise les couleurs pour qu'elle soit comprise entre 0 et 1 :


In [ ]:
X = (china / 255.0).reshape(-1, 3)
print(X.shape)

On dipose de 273 280 points dans un espace de 3 dimensions.

La tâche est de compresser ces $256^3$ couleurs dans un nombre significativement plus petit, à savoir 64 couleurs. Concrètement, on cherche à obtenir 64 clusters dans nos données et recréer par la suite une image similaire en replacant chaque couleur (ou point) par le centre de son cluster le plus proche.

Dans ce cas, on utilise le module ``MiniBatchKMeans``, une version plus sophistiquée qui permet de traiter des volumes de données plus important dans un délais raccourci:

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
# on peut réduire la taille de l'image pour accélerer la convergence
n_colors = 64

X = (china / 255.0).reshape(-1, 3)
    
model = MiniBatchKMeans(n_colors)
labels = model.fit_predict(X)
colors = model.cluster_centers_
new_image = colors[labels].reshape(china.shape)
new_image = (255 * new_image).astype(np.uint8)

# La nouvelle image est créee puis affichée
with plt.style.context('seaborn-white'):
    plt.figure()
    plt.imshow(china)
    plt.title('input: 16 million colors')

    plt.figure()
    plt.imshow(new_image)
    plt.title('{0} colors'.format(n_colors))

On compare l'image d'origine avec l'image compressée. On a reduit le nombre de couleur, initialement égal à $256^3$, à uniquement 64 couleurs. Le résultat reste très satisfaisant !
